<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-pre-processing" data-toc-modified-id="Data-pre-processing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data pre-processing</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Check-missing-data" data-toc-modified-id="Check-missing-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check missing data</a></span></li><li><span><a href="#Number-of-users-in-each-file" data-toc-modified-id="Number-of-users-in-each-file-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Number of users in each file</a></span></li><li><span><a href="#Get-profile-report-for-notifications_allowed" data-toc-modified-id="Get-profile-report-for-notifications_allowed-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Get profile report for notifications_allowed</a></span></li><li><span><a href="#Merge-files" data-toc-modified-id="Merge-files-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Merge files</a></span></li><li><span><a href="#Total-number-of-observations" data-toc-modified-id="Total-number-of-observations-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Total number of observations</a></span></li><li><span><a href="#Split-data-according-to-which-group-of-users-belong" data-toc-modified-id="Split-data-according-to-which-group-of-users-belong-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Split data according to which group of users belong</a></span></li><li><span><a href="#What-features-(columns)-do-we-have?" data-toc-modified-id="What-features-(columns)-do-we-have?-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>What features (columns) do we have?</a></span></li></ul></li><li><span><a href="#Optimization-metrics" data-toc-modified-id="Optimization-metrics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optimization metrics</a></span><ul class="toc-item"><li><span><a href="#Hit-rate" data-toc-modified-id="Hit-rate-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hit rate</a></span></li><li><span><a href="#Retention-rate" data-toc-modified-id="Retention-rate-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Retention rate</a></span></li></ul></li><li><span><a href="#Save-dataframes" data-toc-modified-id="Save-dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save dataframes</a></span></li></ul></div>

# Data preparation

### Data pre-processing

In [1]:
import pandas as pd # used for data storage
import numpy as np # used for data calculations
import pandas_profiling as profil # used for data summary

pd.set_option('display.max_columns', 500) # maximum number of columns to be shown
pd.set_option('display.max_rows', 500) # maximum number of rows to be shown 

#### Load data

In [2]:
notification_allowed = pd.read_csv("notification_allowed.csv")
user_history = pd.read_csv("user_history.csv")

#### Check missing data

In [3]:
notification_allowed.isnull().values.any(), user_history.isnull().values.any()

(False, False)

#### Number of users in each file

In [4]:
len(notification_allowed.login_id.unique()), len(user_history.login_id.unique())

(3859, 4979)

#### Get profile report for notifications_allowed

In [5]:
notification_profile = profil.ProfileReport(notification_allowed, 
                                     title="Allowed Notification Report")

#### Merge files
__user_history__ contains more users than __notification_allowed__, so this two dataframes needs
to be merged for further AB test analysis 

In [6]:
user_data = notification_allowed.merge(user_history, on=['login_id', 'date_id'], how="outer")

#### Total number of observations

In [7]:
len(user_data)

122459

#### Split data according to which group of users belong
- Add column 'group' (for control and test group, in this task group_1 and group_2)

- group_1 consists of users with even login_id

- group_2 consists of users with odd login_id

In [8]:
user_data["group"] = np.nan
user_data.loc[user_data['login_id'] % 2 == 0, "group"] = 'group_1'
user_data.loc[user_data['login_id'] % 2 == 1, "group"] = 'group_2'

#### What features (columns) do we have?

In [9]:
user_data.columns

Index(['login_id', 'time', 'allowed_notifications', 'date_id',
       'registration_date_id', 'registration_channel', 'registration_country',
       'payer', 'dau', 'sessions_count', 'playtime', 'last_login_day',
       'days_active_last_7_days', 'cohort_size', 'elo_rating', 'arena_level',
       'gold', 'runes_stash_class1_tier1', 'runes_stash_class2_tier1',
       'runes_stash_class3_tier1', 'runes_stash_tier2', 'spell_runes_stash',
       'hero_dust', 'item_dust', 'gold_gained_total', 'gold_bought_total',
       'gold_spent_total', 'runes_gained_total_class1_tier1',
       'runes_gained_total_class2_tier1', 'runes_gained_total_class3_tier1',
       'runes_gained_total_tier_2', 'spell_runes_gained_total',
       'runes_spent_class1_tier1', 'runes_spent_class2_tier1',
       'runes_spent_class3_tier1', 'runes_spent_total_tier2',
       'spell_runes_spent_total', 'hero_dust_spent_total',
       'hero_dust_gained_total', 'tokens', 'tokens_gained_total',
       'enter_queue_count', 'batt

### Optimization metrics

#### Hit rate
- Use only neccessary columns for hit rate processing

- Remove duplicated rows

- Remove row for users who have more than 1 row (2 rows).

(Note: The last task is because of outer merging. If one user appears in 2 rows, that means user have NaN and True or False in 'allowed_notification' column. In that case, row with NaN is redundant, so it should be removed)

In [13]:
users_hit_rate = user_data[["login_id", "allowed_notifications", "battles_played_total",
                            "battles_played", "payer", "date_id", "dau", "group"]].drop_duplicates()
duplicated_users = users_hit_rate.groupby("login_id").agg("login_id").count().sort_values(ascending=False)
duplicated_users = duplicated_users[duplicated_users == 2].index

users_hit_rate.drop(users_hit_rate[(users_hit_rate["login_id"].isin(duplicated_users)) & 
                                   (users_hit_rate["allowed_notifications"].isnull())].index,
                   inplace=True)

#### Retention rate
- Add column that is neccessary for calculating retention

- Use only neccessary columns for retention rate processing

In [11]:
user_data['retention'] = user_data['date_id'] - user_data['registration_date_id']
users_retention_rate = user_data[["login_id", "retention", "dau", "group"]]

### Save dataframes

In [14]:
user_data.to_csv('user_data.csv', index=False)
users_hit_rate.to_csv('users_hit_rate.csv', index=False)
users_retention_rate.to_csv('users_retention_rate.csv', index=False)